# Roe Approximate Riemann Solver Shallow Water Equations

$$
\renewcommand{\v}[1]{\mathbf{#1}}
% redefine integral commands with inputs
% indefinite integral without dx
\let\oldint\int
\newcommand{\inta}[1]{\oldint\!\paren{#1}}
\newcommand{\intna}[1]{\oldint\!{#1}}
\def\int{\@ifstar{\inta}{\intna}}
% indefinite integral with dx
\newcommand{\intta}[2]{\oldint\!\paren{#1}\,\mathrm{d}{#2}}
\newcommand{\inttna}[2]{\oldint\!{#1}\,\mathrm{d}{#2}}
\def\intt{\@ifstar{\intta}{\inttna}}
% definite integral with bounds but no dx
\newcommand{\dintna}[3]{\oldint_{#1}^{#2}\!{#3}}
\newcommand{\dinta}[3]{\oldint_{#1}^{#2}\!\paren{#3}}
\def\dint{\@ifstar{\dinta}{\dintna}}
% definite integral
\newcommand{\dinttna}[4]{\oldint_{#1}^{#2}\!{#3}\,\mathrm{d}{#4}}
\newcommand{\dintta}[4]{\oldint_{#1}^{#2}\!\paren{#3}\,\mathrm{d}{#4}}
\def\dintt{\@ifstar{\dintta}{\dinttna}}
\let\oldiint\iint
\newcommand{\iinta}[1]{\oldiint\!\paren{#1}}
\newcommand{\iintna}[1]{\oldiint\!{#1}}
\def\iint{\@ifstar{\iinta}{\iintna}}
% indefinite integral with dx
\newcommand{\iintta}[2]{\oldiint\!\paren{#1}\,\mathrm{d}{#2}}
\newcommand{\iinttna}[2]{\oldiint\!{#1}\,\mathrm{d}{#2}}
\def\iintt{\@ifstar{\iintta}{\iinttna}}
% definite integral with bounds but no dx
\newcommand{\diintna}[3]{\oldiint_{#1}^{#2}\!{#3}}
\newcommand{\diinta}[3]{\oldiint_{#1}^{#2}\!\paren{#3}}
\def\diint{\@ifstar{\diinta}{\diintna}}
% definite integral
\newcommand{\diinttna}[4]{\oldiint_{#1}^{#2}\!{#3}\,\mathrm{d}{#4}}
\newcommand{\diintta}[4]{\oldiint_{#1}^{#2}\!\paren{#3}\,\mathrm{d}{#4}}
\def\diintt{\@ifstar{\diintta}{\diinttna}}
% define macro for bounds
\newcommand{\evalb}[3]{\left. #1 \right\rvert_{#2}^{#3}}
\newcommand{\evall}[2]{\left. #1 \right\rvert_{#2}}
\def\eval{\@ifstar{\evalb}{\evall}}
$$

Conservation Law
$$ \v{q}_t + \v{f}(\v{q})_x = 0 $$


In [1]:
load("GeneralizedShallowWaterVariables.sage")

## Standard Shallow Water Equations

In [2]:
set_generalized_shallow_water_variables(0)
# path substitution
xi = var('xi')
path_subs = {z_1: Z_1_im1 + (Z_1_i - Z_1_im1) * xi, z_2: Z_2_im1 + (Z_2_i - Z_2_im1) * xi}

# integrate over path
C = f_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
B = q_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
A = C*B.inverse()

assert(A[0, 0] == 0)
assert(A[0, 1] == 1)
# A[1, 1] = 2 u_bar
u_bar = (1/2*A[1, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
# A[1, 0] = gh - u^2
h_bar = ((A[1, 0] + u_bar^2)/g).subs(Z_i_to_P_i).subs(Z_im1_to_P_im1).simplify_full()

# check property A*(Q_i - Q_im1) == f_q(Q_i) - f_q(Q_im1)
test = (A.subs(Z_i_to_Q_i).subs(Z_im1_to_Q_im1)*(Q_i - Q_im1) - (f_q.subs(q_to_Q_i) - f_q.subs(q_to_Q_im1))).simplify_full()
assert(test[0] == 0)
assert(test[1] == 0)
print("h_bar")
print(h_bar)
print("u_bar")
print(u_bar)

h_bar
1/2*H_i + 1/2*H_im1
u_bar
(sqrt(H_i)*U_i + sqrt(H_im1)*U_im1)/(sqrt(H_i) + sqrt(H_im1))


## Linear Profile 1 Moment

In [3]:
set_generalized_shallow_water_variables(1)

# path substitution
xi = var('xi')
path_subs = {z_1: Z_1_im1 + (Z_1_i - Z_1_im1) * xi, z_2: Z_2_im1 + (Z_2_i - Z_2_im1) * xi, z_3: Z_3_im1 + (Z_3_i - Z_3_im1) * xi}

# integrate over path
C = f_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
B = q_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
A = C*B.inverse()

# f_q'(q(p)) =
#[                  0                   1                   0]
#[g*h - 1/3*s^2 - u^2                 2*u               2/3*s]
#[             -2*s*u                 2*s                 2*u]
# find Roe averages and assert that A = f_q(q_bar) at least initially
assert(A[0, 0] == 0)
assert(A[0, 1] == 1)
assert(A[0, 2] == 0)
u_bar = (1/2*A[1, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
s_bar = (3/2*A[1, 2].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
h_bar = ((A[1,0].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) + u_bar^2 + 1/3*s_bar^2)/g).simplify_full()
assert((A[2, 0].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) + 2*s_bar*u_bar).simplify_full() == 0)
assert(A[2, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 2*s_bar == 0)
assert(A[2, 2].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 2*u_bar == 0)

# Modify A to match quasilinear form with nonconservative term
# Change A[2, 2] to u instead of 2*u
A[2, 2] = 1/2*A[2, 2]

# check property A*(Q_i - Q_im1) == f_q(Q_i) - f_q(Q_im1)
test = (A.subs(Z_i_to_Q_i).subs(Z_im1_to_Q_im1)*(Q_i - Q_im1) - (f_q.subs(q_to_Q_i) - f_q.subs(q_to_Q_im1))).simplify_full()
assert(test[0] == 0)
assert(test[1] == 0)
# third equation wont satisfy this property as q_3 is not conserved
print("h_bar")
print(h_bar)
print("u_bar")
print(u_bar)
print("s_bar")
print(s_bar)

h_bar
1/2*H_i + 1/2*H_im1
u_bar
(sqrt(H_i)*U_i + sqrt(H_im1)*U_im1)/(sqrt(H_i) + sqrt(H_im1))
s_bar
(sqrt(H_i)*S_i + sqrt(H_im1)*S_im1)/(sqrt(H_i) + sqrt(H_im1))


## 2 Moments/Quadratic Profile

In [16]:
set_generalized_shallow_water_variables(2)

# path substitution
xi = var('xi')
path_subs = {z_1: Z_1_im1 + (Z_1_i - Z_1_im1) * xi, 
             z_2: Z_2_im1 + (Z_2_i - Z_2_im1) * xi, 
             z_3: Z_3_im1 + (Z_3_i - Z_3_im1) * xi, 
             z_4: Z_4_im1 + (Z_4_i - Z_4_im1) * xi}

# integrate over path
C = f_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
B = q_z_j.subs(path_subs).apply_map(lambda e: integrate(e, xi, 0, 1))
A = C*B.inverse()

# f_q'(q(p)) =
#[                            0                             1                             0                             0]
#[g*h - 1/5*k^2 - 1/3*s^2 - u^2                           2*u                         2/3*s                         2/5*k]
#[             -4/5*k*s - 2*s*u                           2*s                   4/5*k + 2*u                         4/5*s]
#[   -2/7*k^2 - 2/3*s^2 - 2*k*u                           2*k                         4/3*s                   4/7*k + 2*u]
# find Roe averages and assert that A = f_q(q_bar) at least initially
assert(A[0, 0] == 0)
assert(A[0, 1] == 1)
assert(A[0, 2] == 0)
assert(A[0, 3] == 0)
u_bar = (1/2*A[1, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
s_bar = (3/2*A[1, 2].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
k_bar = (5/2*A[1, 3].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1)).simplify_full()
h_bar = ((A[1,0].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) + u_bar^2 + 1/3*s_bar^2 + 1/5*k_bar^2)/g).simplify_full()
assert((A[2, 0].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) + 2*s_bar*u_bar + 4/5*k_bar*s_bar).simplify_full() == 0)
assert(A[2, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 2*s_bar == 0)
assert(A[2, 2].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 2*u_bar - 4/5*k_bar == 0)
assert(A[2, 3].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 4/5*s_bar == 0)
assert(A[3, 0].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) + 2/7*k_bar^2 + 2/3*s_bar^2 + 2*k_bar*u_bar == 0)
assert(A[3, 1].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 2*k_bar == 0)
assert(A[3, 2].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 4/3*s_bar == 0)
assert(A[3, 3].subs(Z_i_to_P_i).subs(Z_im1_to_P_im1) - 4/7*k_bar - 2*u_bar == 0)

# Modify A to match quasilinear form with nonconservative term
# Change A[2, 2] to u instead of 2*u
A = A - Q_p.subs({u: u_bar, s: s_bar, k: k_bar}).subs(P_i_to_Z_i).subs(P_im1_to_Z_im1)

# check property A*(Q_i - Q_im1) == f_q(Q_i) - f_q(Q_im1)
test = (A.subs(Z_i_to_Q_i).subs(Z_im1_to_Q_im1)*(Q_i - Q_im1) - (f_q.subs(q_to_Q_i) - f_q.subs(q_to_Q_im1))).simplify_full()
assert(test[0] == 0)
assert(test[1] == 0)
# third and fourth equation wont satisfy this property as q_3 and q_4 is not conserved
print("h_bar")
print(h_bar)
print("u_bar")
print(u_bar)
print("s_bar")
print(s_bar)
print("k_bar")
print(k_bar)

h_bar
1/2*H_i + 1/2*H_im1
u_bar
(sqrt(H_i)*U_i + sqrt(H_im1)*U_im1)/(sqrt(H_i) + sqrt(H_im1))
s_bar
(sqrt(H_i)*S_i + sqrt(H_im1)*S_im1)/(sqrt(H_i) + sqrt(H_im1))
k_bar
(sqrt(H_i)*K_i + sqrt(H_im1)*K_im1)/(sqrt(H_i) + sqrt(H_im1))
